In [2]:
import numpy as np
import random
from pathlib import Path
from LTTraceData import LTTraceData
from PyLTSpice import SimCommander, RawRead, SimRunner, SpiceEditor
import CoilcraftRandomSelect as crs
import MurataRandomSelect as mrs

high_side = True
low_side = False

# crs.closest_coilcraft_inductor(46E-6)
# mrs.closest_murata_capacitor(0.99E-6)
print(crs.indexed_coilcraft_inductor(3136)["SubCkt"])
print(mrs.indexed_murata_capacitor("C1", 11464)["SubCkt"])

def frequencytoltpulse(frequency, side: bool):
    deadband = 8  # ns
    period = 1 / (frequency / 1E9)  # Convert to Gigahertz to ensure the period will be in nanoseconds
    if side == low_side:
        return f"PULSE(3.3 0 0n 1n 1n {period + (deadband * 2)}n {period * 2}n)"
    else:
        return f"PULSE(0 3.3 {deadband}n 1n 1n {period}n {period * 2}n)"

print(frequencytoltpulse(100E3, high_side))
print(frequencytoltpulse(100E3, low_side))

Using LTspice installed in : 'C:\Users\treys\AppData\Local\Programs\ADI\LTspice\LTspice.exe' 


IndexError: single positional indexer is out-of-bounds

In [4]:
filename = r'LTFiles/_I-103_C1-362_C2-444_C3-323_C4-625.raw'
ltr = RawRead(filename, verbose=False)

v_in = LTTraceData(ltr, 'V(vin)')
v_out = LTTraceData(ltr, 'V(vout)')
i_in = LTTraceData(ltr, 'I(Vs)', invert = True)
i_out = LTTraceData(ltr, 'I(R1)')

if v_out.peak_to_peak > 2:
    print("Peak to peak error")

if v_out.min < 4 or v_out.max > 8:
    print("Limit error")

print(f'Vout | Average: {v_out.average}')
print(f'Vin | Average: {v_in.average}')
print(f'Iout | Average: {i_out.average}')
print(f'Iin | Average: {i_in.average}')

print((v_out.average * i_out.average) / (v_in.average * i_in.average))

Vout | Average: 4.981719017028809
Vin | Average: 10.0
Iout | Average: 0.996343731880188
Iin | Average: 1.2984588146209717
0.38226122081922664


In [ ]:
def frequencytoltpulse(frequency, side: bool):
    deadband = 8  # ns
    period = 1 / (frequency / 1E9)  # Convert to Gigahertz to ensure the period will be in nanoseconds
    if side == low_side:
        return f"PULSE(3.3 0 0n 1n 1n {period + (deadband * 2)}n {period * 2}n)"
    else:
        return f"PULSE(0 3.3 {deadband}n 1n 1n {period}n {period * 2}n)"


fsw = 100E3
L = 10E-6
C = 47E-6

# select spice model
# LTC = SimCommander("LTFiles/EPC23102_Mine.asc")

v_out_max = 999
v_out_min = 0

random.seed(256)

while v_out_max > 6 or v_out_min < 4:
    LTC = SimRunner(output_folder='LTFiles')
    # LTC.create_netlist('LTFiles/EPC23102_Mine.asc')
    netlist = SpiceEditor('LTFiles/EPC23102_Mine.asc')
    # set default arguments
    netlist.set_component_value('VHSin', frequencytoltpulse(fsw, high_side))
    netlist.set_component_value('VLSin', frequencytoltpulse(fsw, low_side))
    inductor = crs.random_coilcraft_inductor()
    print(inductor['PartNumber'])
    netlist.add_instruction(inductor['SubCkt'])
    # capacitor = mrs.random_murata_capacitor('C1')
    # print(capacitor['PartNumber'])
    # netlist.add_instruction(capacitor['SubCkt'])

    netlist.add_instruction(mrs.random_murata_capacitor('C1')['SubCkt'])
    netlist.add_instruction(mrs.random_murata_capacitor('C2')['SubCkt'])
    netlist.add_instruction(mrs.random_murata_capacitor('C3')['SubCkt'])
    netlist.add_instruction(mrs.random_murata_capacitor('C4')['SubCkt'])

    # print(netlist.get_components())

    netlist.write_netlist(Path("LTFiles/FullyCustom.net"))
    raw, log = LTC.run_now(netlist, run_filename="FullyCustom.net")

    LTR = RawRead('LTFiles/FullyCustom.raw')
    # print(LTR.get_trace_names())

    v_out = LTTraceData(LTR, 'V(vout)')
    print(v_out.min)
    print(v_out.max)
    print(v_out.average)
    v_out_max = v_out.max
    v_out_min = v_out.min

# print(raw)
# print(log)

# # run_netlist_file = f'LTFiles/F{fsw}_L{L}_C{C}'
# # run_netlist_file = f'LTFiles/FullyRandom'
# # LTC.run(run_filename=f'{run_netlist_file}.net')

In [18]:
import pandas as pd
inductors = pd.read_csv('CoilcraftInductors.csv')  # Get all the inductors from the file
inductors = inductors[inductors['PartNumber'].str.contains("0201|0402|0603|0805|1008|1206|1210") == False]
inductors = inductors[inductors['Inductance'] > 1E-6].reset_index(drop=True)
inductors

,Unnamed: 0,PartNumber,Inductance,SubCkt
0,1504,XEL3520-112,0.000001,*=============================================...
1,1505,MVR1261C-112,0.000001,*=============================================...
2,1506,XAL6020-112,0.000001,*=============================================...
3,1507,XAL5020-122,0.000001,*=============================================...
4,1508,XAL7070-122,0.000001,*=============================================...
...,...,...,...,...
1694,3404,LPS5050-106,0.010000,*=============================================...
1695,3406,LPS6225-106,0.010000,*=============================================...
1696,3407,HA3778,7.700000,*=============================================...
1697,3408,GA3199,19.000000,*=============================================...
